In [1]:
import math
import copy
import random

In [2]:
class GaussianSolver:

    def __init__(self):
        self.operations_count = 0
    def reset_counter(self):
        self.operations_count = 0
    def add_operations(self, count=1):
        self.operations_count += count

    "LU разложение матрицы"
    def lu_decomposition(self, A):
        self.reset_counter()
        n = len(A)
        L = [[0.0] * n for _ in range(n)]
        U = copy.deepcopy(A)
        for i in range(n):
            L[i][i] = 1.0
            self.add_operations(1)
        for k in range(n-1):
            for i in range(k+1, n):
                if abs(U[k][k]) < 1e-15:
                    raise ValueError("Нулевой диагональный элемент")

                multiplier = U[i][k] / U[k][k]
                L[i][k] = multiplier
                self.add_operations(1)

                for j in range(k, n):
                    U[i][j] -= multiplier * U[k][j]
                    self.add_operations(2)
        return L, U

    "LUP разложение матрицы"
    def lup_decomposition(self, A):
        self.reset_counter()
        n = len(A)
        L = [[0.0] * n for _ in range(n)]
        U = copy.deepcopy(A)
        P = [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]
        for i in range(n):
            L[i][i] = 1.0
            self.add_operations(1)
        for k in range(n-1):
            max_row = k
            for i in range(k+1, n):
                if abs(U[i][k]) > abs(U[max_row][k]):
                    max_row = i
                self.add_operations(2)
            if max_row != k:
                U[k], U[max_row] = U[max_row], U[k]
                P[k], P[max_row] = P[max_row], P[k]
                for j in range(k):
                    L[k][j], L[max_row][j] = L[max_row][j], L[k][j]
                self.add_operations(3)
            for i in range(k+1, n):
                if abs(U[k][k]) < 1e-15:
                    raise ValueError("Нулевой диагональный элемент")
                multiplier = U[i][k] / U[k][k]
                L[i][k] = multiplier
                self.add_operations(1)
                for j in range(k, n):
                    U[i][j] -= multiplier * U[k][j]
                    self.add_operations(2)
        return L, U, P

    "решение через LU"
    def solve_lu(self, L, U, b):
        self.reset_counter()
        n = len(L)
        y = [0.0] * n
        for i in range(n):
            sum_ly = 0.0
            for j in range(i):
                sum_ly += L[i][j] * y[j]
                self.add_operations(2)
            y[i] = (b[i] - sum_ly) / L[i][i]
            self.add_operations(2)
        x = [0.0] * n
        for i in range(n-1, -1, -1):
            sum_ux = 0.0
            for j in range(i+1, n):
                sum_ux += U[i][j] * x[j]
                self.add_operations(2)
            x[i] = (y[i] - sum_ux) / U[i][i]
            self.add_operations(2)
        return x

    "решаем через LUP"
    def solve_lup(self, L, U, P, b):
        n = len(P)
        Pb = [0.0] * n
        for i in range(n):
            for j in range(n):
                if P[i][j] == 1.0:
                    Pb[i] = b[j]
                    self.add_operations(1)
                    break
        return self.solve_lu(L, U, Pb)

    "вычисления определителя U матрицы"
    def determinant_from_lu(self, U):
        det = 1.0
        for i in range(len(U)):
            det *= U[i][i]
            self.add_operations(1)
        return det

    "просто метод Гаусса"
    def solve_direct(self, A, b, pivot_type='none'):
        self.reset_counter()
        n = len(A)
        A = copy.deepcopy(A)
        b = copy.deepcopy(b)
        for k in range(n-1):
            if pivot_type == 'column':
                max_row = k
                for i in range(k+1, n):
                    if abs(A[i][k]) > abs(A[max_row][k]):
                        max_row = i
                    self.add_operations(2)
                if max_row != k:
                    A[k], A[max_row] = A[max_row], A[k]
                    b[k], b[max_row] = b[max_row], b[k]
                    self.add_operations(2)
            for i in range(k+1, n):
                multiplier = A[i][k] / A[k][k]
                self.add_operations(1)
                for j in range(k+1, n):
                    A[i][j] -= multiplier * A[k][j]
                    self.add_operations(2)
                b[i] -= multiplier * b[k]
                self.add_operations(2)

        x = [0.0] * n
        for i in range(n-1, -1, -1):
            sum_ax = 0.0
            for j in range(i+1, n):
                sum_ax += A[i][j] * x[j]
                self.add_operations(2)
            x[i] = (b[i] - sum_ax) / A[i][i]
            self.add_operations(2)
        return x

In [3]:
def print_matrix(matrix, name):
    print(f"{name} = ")
    for row in matrix:
        print("  [" + ", ".join(f"{x:8.4f}" for x in row) + "]")

def generate_random_vector(n, a=-10, b=10):
    return [random.uniform(a, b) for _ in range(n)]

def test_lu_decomposition():
    print("=" * 60)
    print("ТЕСТ LU-РАЗЛОЖЕНИЯ")
    print("=" * 60)

    solver = GaussianSolver()

    A = [
        [2.0, 1.0, -1.0],
        [-3.0, -1.0, 2.0],
        [-2.0, 1.0, 2.0]
    ]
    b = [8.0, -11.0, -3.0]

    print("Исходная система:")
    print_matrix(A, "A")
    print(f"b = {b}")

    print("\n1. LU-разложение:")
    L, U = solver.lu_decomposition(A)
    print_matrix(L, "L")
    print_matrix(U, "U")

    print("\nПроверка A = L * U:")
    n = len(A)
    LU = [[0.0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            for k in range(n):
                LU[i][j] += L[i][k] * U[k][j]
    print_matrix(LU, "L*U")

    print("\nРешение через LU:")
    x_lu = solver.solve_lu(L, U, b)
    print(f"x = {[f'{val:.4f}' for val in x_lu]}")
    print(f"Операций: {solver.operations_count}")

    det = solver.determinant_from_lu(U)
    print(f"Определитель: {det:.4f}")

def test_lup_decomposition():
    print("\n" + "=" * 60)
    print("ТЕСТ LUP-РАЗЛОЖЕНИЯ")
    print("=" * 60)

    solver = GaussianSolver()

    A = [
        [0.0, 1.0, 2.0],
        [3.0, 4.0, 5.0],
        [6.0, 7.0, 9.0]
    ]
    b = [1.0, 2.0, 3.0]

    print("Исходная система (требует перестановок):")
    print_matrix(A, "A")
    print(f"b = {b}")

    L, U, P = solver.lup_decomposition(A)
    print_matrix(L, "L")
    print_matrix(U, "U")
    print_matrix(P, "P")

    x_lup = solver.solve_lup(L, U, P, b)
    print(f"Решение через LUP: x = {[f'{val:.4f}' for val in x_lup]}")
    print(f"Операций: {solver.operations_count}")

def test_multiple_rhs():
    print("\n" + "=" * 60)
    print("ТЕСТ С РАЗНЫМИ ПРАВЫМИ ЧАСТЯМИ")
    print("=" * 60)

    solver = GaussianSolver()

    A = [
        [4.0, 1.0, 2.0],
        [1.0, 3.0, 1.0],
        [2.0, 1.0, 5.0]
    ]

    print("Матрица A:")
    print_matrix(A, "A")

    L, U = solver.lu_decomposition(A)
    lu_ops = solver.operations_count
    print(f"LU-разложение: {lu_ops} операций")

    num_systems = 3
    total_lu_ops = lu_ops

    print(f"\nРешение {num_systems} систем с разными правыми частями:")

    for i in range(num_systems):
        b = generate_random_vector(3)
        print(f"\nСистема {i+1}: b = {[f'{val:.4f}' for val in b]}")

        solver.reset_counter()
        x_lu = solver.solve_lu(L, U, b)
        lu_solve_ops = solver.operations_count
        total_lu_ops += lu_solve_ops
        print(f"  LU решение: {lu_solve_ops} операций")

        solver.reset_counter()
        x_direct = solver.solve_direct(A, b)
        direct_ops = solver.operations_count
        print(f"  Прямое решение: {direct_ops} операций")

        error = math.sqrt(sum((x_lu[j] - x_direct[j])**2 for j in range(3)))
        print(f"  Погрешность: {error:.2e}")

    print(f"\nИтого операций для {num_systems} систем:")
    print(f"  С LU-разложением: {total_lu_ops}")
    print(f"  Без LU-разложения: {direct_ops * num_systems}")

def compare_operations():
    print("\n" + "=" * 60)
    print("СРАВНЕНИЕ ЭФФЕКТИВНОСТИ LU-РАЗЛОЖЕНИЯ")
    print("=" * 60)
    solver = GaussianSolver()
    sizes = [3, 5, 7]
    solutions_counts = [1, 2, 5, 10]
    for n in sizes:
        print(f"\n--- Матрица {n}x{n} ---")
        A = [[random.uniform(-10, 10) for _ in range(n)] for _ in range(n)]
        solver.reset_counter()
        L, U = solver.lu_decomposition(A)
        lu_decomp_cost = solver.operations_count
        b_test = [random.uniform(-10, 10) for _ in range(n)]
        solver.reset_counter()
        solver.solve_lu(L, U, b_test)
        lu_solve_once = solver.operations_count
        solver.reset_counter()
        solver.solve_direct(A, b_test)
        direct_once = solver.operations_count
        print(f"LU разложение: {lu_decomp_cost} операций")
        print(f"Одно LU решение: {lu_solve_once} операций")
        print(f"Одно прямое решение: {direct_once} операций")
        print(f"\n{'Кол-во реш.':<12} {'LU всего':<12} {'Прямых всего':<12} {'Выгода':<12} {'Результат':<20}")
        print("-" * 70)
        for k in solutions_counts:
            total_lu = lu_decomp_cost + k * lu_solve_once
            total_direct = k * direct_once
            advantage = total_direct - total_lu
            if advantage > 0:
                percent = advantage / total_direct * 100
                result = f"ВЫГОДА {percent:.1f}%"
                advantage_str = f"+{advantage}"
            else:
                result = "не выгодно"
                advantage_str = f"{advantage}"
            print(f"{k:<12} {total_lu:<12} {total_direct:<12} {advantage_str:<12} {result:<20}")
        if lu_solve_once < direct_once:
            denominator = direct_once - lu_solve_once
            if denominator > 0:
                break_even = lu_decomp_cost / denominator
                print(f"\nТочка безубыточности: {break_even:.1f} решений")
                print(f"→ LU выгодно при {math.ceil(break_even)}+ решениях")

In [4]:
def main():
    test_lu_decomposition()
    test_lup_decomposition()
    test_multiple_rhs()
    compare_operations()

if __name__ == "__main__":
    main()

ТЕСТ LU-РАЗЛОЖЕНИЯ
Исходная система:
A = 
  [  2.0000,   1.0000,  -1.0000]
  [ -3.0000,  -1.0000,   2.0000]
  [ -2.0000,   1.0000,   2.0000]
b = [8.0, -11.0, -3.0]

1. LU-разложение:
L = 
  [  1.0000,   0.0000,   0.0000]
  [ -1.5000,   1.0000,   0.0000]
  [ -1.0000,   4.0000,   1.0000]
U = 
  [  2.0000,   1.0000,  -1.0000]
  [  0.0000,   0.5000,   0.5000]
  [  0.0000,   0.0000,  -1.0000]

Проверка A = L * U:
L*U = 
  [  2.0000,   1.0000,  -1.0000]
  [ -3.0000,  -1.0000,   2.0000]
  [ -2.0000,   1.0000,   2.0000]

Решение через LU:
x = ['2.0000', '3.0000', '-1.0000']
Операций: 24
Определитель: -1.0000

ТЕСТ LUP-РАЗЛОЖЕНИЯ
Исходная система (требует перестановок):
A = 
  [  0.0000,   1.0000,   2.0000]
  [  3.0000,   4.0000,   5.0000]
  [  6.0000,   7.0000,   9.0000]
b = [1.0, 2.0, 3.0]
L = 
  [  1.0000,   0.0000,   0.0000]
  [  0.0000,   1.0000,   0.0000]
  [  0.5000,   0.5000,   1.0000]
U = 
  [  6.0000,   7.0000,   9.0000]
  [  0.0000,   1.0000,   2.0000]
  [  0.0000,   0.0000,  -0.5000